In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
from tqdm import tqdm
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

In [3]:
# 数据准备
train = pd.read_csv('../data/train_set.csv')
test = pd.read_csv('../data/test_set.csv')

data = pd.concat([train,test])
print(data.shape[0])
data.head(3)

36169


,ID,age,balance,campaign,contact,day,default,duration,education,housing,job,loan,marital,month,pdays,poutcome,previous,y
0,1,43,291,2,unknown,9,no,150,tertiary,yes,management,no,married,may,-1,unknown,0,0.0
1,2,42,5076,1,cellular,7,no,99,primary,yes,technician,no,divorced,apr,251,other,2,0.0
2,3,47,104,2,cellular,14,no,77,secondary,yes,admin.,yes,married,jul,-1,unknown,0,0.0


In [4]:
# 查看正负样本情况
train[train['y']==0].shape[0],train[train['y']==1].shape[0]

(22356, 2961)

# 特征工程

In [10]:
## 特征提取
feature = data.columns.tolist()
feature.remove('ID')
feature.remove('y')
feature

['age',
 'balance',
 'campaign',
 'contact',
 'day',
 'default',
 'duration',
 'education',
 'housing',
 'job',
 'loan',
 'marital',
 'month',
 'pdays',
 'poutcome',
 'previous']

In [12]:
data['age'].unique()

array([43, 42, 47, 28, 56, 32, 33, 55, 54, 59, 48, 53, 30, 39, 31, 49, 25,
       44, 38, 35, 40, 29, 41, 57, 27, 46, 50, 36, 61, 21, 24, 26, 37, 45,
       51, 52, 34, 58, 74, 60, 22, 23, 79, 80, 75, 66, 20, 81, 71, 69, 83,
       73, 72, 78, 65, 70, 63, 67, 19, 64, 89, 77, 62, 85, 76, 68, 82, 86,
       84, 18, 87, 95, 92, 90, 93, 94, 88], dtype=int64)

In [ ]:
sparse_feature = ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature = list(set(feature)-set(sparse_feature))

In [ ]:
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name



ll = []
for f in ['campaign', 'contact','default','education','housing','job','loan','marital','poutcome']:
    data,_ = feature_count(data,['day','month',f])
    ll.append(_)
    
def get_new_columns(name,aggs):
    l = []
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg))=="<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l

for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    for den in dense_feature:
        aggs[den]=['mean','max','min','std']
    aggs.pop(d)
    temp = data.groupby(d).agg(aggs).reset_index()
    temp.columns = [d]+get_new_columns(d,aggs)
    data = pd.merge(data,temp,on=d,how='left')
    
for s in ['contact','default','education','housing','job','loan','marital','month','poutcome']:
    data = pd.concat([data,pd.get_dummies(data[s],prefix=s+'_')],axis=1)
    data.drop(s,axis=1,inplace=True)
